In [11]:
class HuffmanNode:
    def __init__(self, word_id, frequency):
        self.word_id = word_id
        self.frequency = frequency
        self.left_child = None
        self.right_child = None
        self.father = None
        self.Huffman_code = []
        self.path = []


class HuffmanTree:
    def __init__(self, wordid_frequency_dict):
        self.word_count = len(wordid_frequency_dict)
        self.wordid_code = dict()
        self.wordid_path = dict()
        self.root = None
        unmerge_node_list = [HuffmanNode(wordid, frequency) for wordid, frequency in wordid_frequency_dict.items()] # Unmerged node list
        self.huffman = [HuffmanNode(wordid, frequency) for wordid, frequency in wordid_frequency_dict.items()] # Store all leaf nodes and intermediate nodes
        # Build huffman tree
        self.build_tree(unmerge_node_list)
        # Generate huffman code
        self.generate_huffman_code_and_path()

    def merge_node(self, node1, node2):
        sum_frequency = node1.frequency + node2.frequency
        mid_node_id = len(self.huffman)
        father_node = HuffmanNode(mid_node_id, sum_frequency)
        if node1.frequency >= node2.frequency:
            father_node.left_child = node1
            father_node.right_child = node2
        else:
            father_node.left_child = node2
            father_node.right_child = node1
        self.huffman.append(father_node)
        return father_node

    def build_tree(self, node_list):
        while len(node_list) > 1:
            i1 = 0  # Node with least frequency
            i2 = 1  # Node with the second smallest frequency
            if node_list[i2].frequency < node_list[i1].frequency:
                [i1, i2] = [i2, i1]
            for i in range(2, len(node_list)):
                if node_list[i].frequency < node_list[i2].frequency:
                    i2 = i
                    if node_list[i2].frequency < node_list[i1].frequency:
                        [i1, i2] = [i2, i1]
            father_node = self.merge_node(node_list[i1], node_list[i2])  # Combine the least frequent two nodes
            if i1 < i2:
                node_list.pop(i2)
                node_list.pop(i1)
            elif i1 > i2:
                node_list.pop(i1)
                node_list.pop(i2)
            else:
                raise RuntimeError('i1 should not be equal to i2')
            node_list.insert(0, father_node)  # Insert new node
        self.root = node_list[0]

    def generate_huffman_code_and_path(self):
        stack = [self.root]
        while len(stack) > 0:
            node = stack.pop()
            while node.left_child or node.right_child:
                code = node.Huffman_code
                path = node.path
                node.left_child.Huffman_code = code + [1]
                node.right_child.Huffman_code = code + [0]
                node.left_child.path = path + [node.word_id]
                node.right_child.path = path + [node.word_id]
                stack.append(node.right_child)
                node = node.left_child
            word_id = node.word_id
            word_code = node.Huffman_code
            word_path = node.path
            self.huffman[word_id].Huffman_code = word_code
            self.huffman[word_id].path = word_path
            # Write the Huffman code and path calculated by the node into the value of the dictionary
            self.wordid_code[word_id] = word_code
            self.wordid_path[word_id] = word_path

    # Get the id of all positive nodes and the id of all negative nodes
    def get_all_pos_and_neg_path(self):
        positive = []  # Array of positive paths of all words
        negative = []  # Array of negative paths for all words
        for word_id in range(self.word_count):
            pos_id = []
            neg_id = []
            for i, code in enumerate(self.huffman[word_id].Huffman_code):
                if code == 1:
                    pos_id.append(self.huffman[word_id].path[i])
                else:
                    neg_id.append(self.huffman[word_id].path[i])
            positive.append(pos_id)
            negative.append(neg_id)
        return positive, negative


def test():
    word_frequency = {0: 4, 1: 6, 2: 3, 3: 2, 4: 2}
    print(word_frequency)
    tree = HuffmanTree(word_frequency)
    print(tree.wordid_code)
    print(tree.wordid_path)
    for i in range(len(word_frequency)):
        print(tree.huffman[i].path)
    print(tree.get_all_pos_and_neg_path())


if __name__ == '__main__':
    test()

{0: 4, 1: 6, 2: 3, 3: 2, 4: 2}
{1: [1, 1], 0: [1, 0], 3: [0, 1, 1], 4: [0, 1, 0], 2: [0, 0]}
{1: [8, 7], 0: [8, 7], 3: [8, 6, 5], 4: [8, 6, 5], 2: [8, 6]}
[8, 7]
[8, 7]
[8, 6]
[8, 6, 5]
[8, 6, 5]
([[8], [8, 7], [], [6, 5], [6]], [[7], [], [8, 6], [8], [8, 5]])


In [12]:
import numpy as np
from collections import deque
import nltk
import re
from nltk.corpus import brown
from nltk.corpus import gutenberg
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('brown')
nltk.download('punkt')


class InputData:
    def __init__(self, sentences):
        self.sentences = sentences
        self.normalize()
        self.counter = 0
        self.wordId_frequency_dict = dict()
        self.word_count = 0  # Number of words (repeated words only count as 1)
        self.word_count_sum = 0  # Total number of words (the number of repeated words also accumulates)
        self.sentence_count = 0  # Number of sentences
        self.id2word_dict = dict()
        self.word2id_dict = dict()
        self._init_dict()  # Initialize the dictionary
        self.huffman_tree = HuffmanTree(self.wordId_frequency_dict)  # Hoffman Tree
        self.huffman_pos_path, self.huffman_neg_path = self.huffman_tree.get_all_pos_and_neg_path()
        self.word_pairs_queue = deque()

        print('Word Count is:', self.word_count)
        print('Word Count Sum is', self.word_count_sum)
        print('Sentence Count is:', self.sentence_count)
        print('Tree Node is:', len(self.huffman_tree.huffman))

    def normalize(self):
      stop_words = nltk.corpus.stopwords.words('english')
      norm_sentences_word_list = []
      for word_list in self.sentences:
        sentence = " ".join(word for word in word_list)
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        sentence = sentence.lower()
        sentence = re.sub(' +', ' ', sentence)
        sentence = sentence.strip()
        norm_word_list = sentence.split(' ')
        norm_word_list = [word for word in norm_word_list if word not in stop_words]
        if(len(norm_word_list) > 1):
          norm_sentences_word_list.append(norm_word_list)
       
      self.sentences = norm_sentences_word_list

    def _init_dict(self):
        word_freq = dict()
        for word_list in self.sentences:
            self.word_count_sum += len(word_list)
            self.sentence_count += 1
            for word in word_list:
                try:
                    word_freq[word] += 1
                except:
                    word_freq[word] = 1
        word_id = 0
        # Initialize word2id_dict, id2word_dict, wordId_frequency_dict dictionary
        for per_word, per_count in word_freq.items():
            self.id2word_dict[word_id] = per_word
            self.word2id_dict[per_word] = word_id
            self.wordId_frequency_dict[word_id] = per_count
            word_id += 1
        self.word_count = len(self.word2id_dict)

    def generate_center_context_pairs(self, window_size):
        self.counter += 1
        if not self.sentences[20*(self.counter-1):20*self.counter]:
            self.counter = 1
            self.word_pairs_queue.clear()
        sub_wids = [[self.word2id_dict[word] for word in word_list] for word_list in self.sentences[20*(self.counter-1):20*self.counter]]


        for words in sub_wids:
          sentence_length = len(words)
          for index, center_word in enumerate(words):
            start = index - window_size
            end = index + window_size + 1

            context_words = []
            for index_2 in range(start,end):
              if 0 <= index_2 < sentence_length and index_2 != index:
                context_words.append(words[index_2])

            self.word_pairs_queue.append((center_word, context_words))           


    def get_batch_pairs(self, batch_size, window_size):

        while len(self.word_pairs_queue) < batch_size:
          self.generate_center_context_pairs(window_size)              
              
        result_pairs = []  # Returns a positive sample pair of mini-batch size
        for _ in range(batch_size):
            result_pairs.append(self.word_pairs_queue.popleft())
        return result_pairs


    def get_pos_neg_pairs(self, pairs):
        neg_word_pair = []
        pos_word_pair = []
        for pair in pairs:
            for context_word in pair[1]:
              pos_word_pair += zip([pair[0]] * len(self.huffman_pos_path[context_word]), self.huffman_pos_path[context_word])
              neg_word_pair += zip([pair[0]] * len(self.huffman_neg_path[context_word]), self.huffman_neg_path[context_word])
        return pos_word_pair, neg_word_pair


    def evaluate_pairs_count(self):
        return self.word_count_sum


def test():
    sentences = brown.sents(categories=['news'])[:2]
    test_data = InputData(sentences)

    print(" ".join(word for word in sentences[0]))
    print(" ".join(word for word in sentences[1]))

    
    batch_pairs = test_data.get_batch_pairs(10, 2)
    pos_pairs, neg_pairs = test_data.get_pos_neg_pairs(batch_pairs)
    batch_word_pairs = []
    for pair in batch_pairs:
        batch_word_pairs.append((test_data.id2word_dict[pair[0]] , [test_data.id2word_dict[i] for i in pair[1]]))
    print(batch_pairs)
    print(batch_word_pairs)
    
    print(test_data.huffman_pos_path[0])
    print(test_data.huffman_neg_path[0])
    print(test_data.huffman_pos_path[1])
    print(test_data.huffman_neg_path[1])
    print(test_data.huffman_pos_path[2])
    print(test_data.huffman_neg_path[2])

    print(pos_pairs)
    print(neg_pairs)


if __name__ == '__main__':
    test()

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Word Count is: 29
Word Count Sum is 34
Sentence Count is: 2
Tree Node is: 57
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .
[(0, [1, 2]), (1, [0, 2, 3]), (2, [0, 1, 3, 4]), (3, [1, 2, 4, 5

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics.pairwise import euclidean_distances


class SkipGramModel(nn.Module):
    def __init__(self, emb_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(2*emb_size-1, emb_dimension, sparse=True)
        self.w_embeddings = nn.Embedding(2*emb_size-1, emb_dimension, sparse=True)
        self._init_emb()

    def _init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.w_embeddings.weight.data.uniform_(-0, 0)

    def forward(self, pos_u, pos_w, neg_u, neg_w):
        pos_u_emb = self.u_embeddings(torch.LongTensor(pos_u))
        pos_w_emb = self.w_embeddings(torch.LongTensor(pos_w))

        neg_u_emb = self.u_embeddings(torch.LongTensor(neg_u))
        neg_w_emb = self.w_embeddings(torch.LongTensor(neg_w))

        score = torch.mul(pos_u_emb, pos_w_emb)
        score = torch.sum(score, dim=1).squeeze()
        score = F.logsigmoid(score)

        neg_score = torch.mul(neg_u_emb, neg_w_emb)
        neg_score = torch.sum(neg_score, dim=1).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)

        # L = log sigmoid (Xw.T * θv) + ∑neg(v) [log sigmoid (-Xw.T * θneg(v))]
        loss = -1 * (torch.sum(score) + torch.sum(neg_score)) 
        return loss

    def distance_matrix(self, word_count):
        embedding = self.u_embeddings.weight.data.numpy()[:word_count]
        distance_matrix = euclidean_distances(embedding)
        return distance_matrix


def test():
    model = SkipGramModel(100, 10)
    id2word = dict()
    for i in range(100):
        id2word[i] = str(i)
    pos_u = [0, 0, 1, 1, 1]
    pos_w = [102, 134, 173, 183, 148]
    neg_u = [0, 0, 0, 1, 1]
    neg_w = [123, 166, 192, 111, 177]
    model.forward(pos_u, pos_w, neg_u, neg_w)


if __name__ == '__main__':
    test()

In [75]:
import torch.optim as optim
from tqdm import tqdm
# from torch.optim.lr_scheduler import LambdaLR

# hyper parameters
WINDOW_SIZE = 2 
BATCH_SIZE = 1000  # mini-batch
EMB_DIMENSION = 100  # embedding dimension
LR = 0.01  # Learning rate


class Word2Vec:
    def __init__(self,sentences):
        self.data = InputData(sentences)
        self.model = SkipGramModel(self.data.word_count, EMB_DIMENSION)
        self.lr = LR
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)
        # lambda1 = lambda epoch: 0.99 ** epoch
        # self.scheduler = SkipGramModel(self.optimizer, lr_lambda=lambda1)

    def train(self):
        print("CBOW Training......")
        pairs_count = self.data.evaluate_pairs_count()
        print("pairs_count", pairs_count)
        batch_count = pairs_count / BATCH_SIZE
        print("batch_count", batch_count)
        for epoch in range(1,51):
            mean_loss = 0
            process_bar = tqdm(range(int(batch_count)))
            for i in process_bar:
                pairs = self.data.get_batch_pairs(BATCH_SIZE, WINDOW_SIZE)
                pos_pairs, neg_pairs = self.data.get_pos_neg_pairs(pairs)

                pos_u = [int(pair[0]) for pair in pos_pairs]
                pos_w = [int(pair[1]) for pair in pos_pairs]
                neg_u = [int(pair[0]) for pair in neg_pairs]
                neg_w = [int(pair[1]) for pair in neg_pairs]

                self.optimizer.zero_grad()
                loss = self.model.forward(pos_u, pos_w, neg_u, neg_w)
                loss.backward()
                self.optimizer.step()
                mean_loss += loss

            print("epoch:",epoch,"loss:",mean_loss/int(batch_count))
            # self.scheduler.step()

    def get_distance_matrix(self):
        distance_matrix = self.model.distance_matrix(self.data.word_count)
        return distance_matrix


In [76]:
sentences = brown.sents(categories=['news','reviews','government','hobbies','romance'])
w2v = Word2Vec(sentences)

Word Count is: 24616
Word Count Sum is 170964
Sentence Count is: 17106
Tree Node is: 49231


In [77]:
w2v.train()

CBOW Training......
pairs_count 170964
batch_count 170.964


100%|██████████| 170/170 [00:21<00:00,  7.78it/s]


epoch: 1 loss: tensor(30116.2832, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:22<00:00,  7.40it/s]


epoch: 2 loss: tensor(30063.9785, grad_fn=<DivBackward0>)


...

epoch: 48 loss: tensor(21262.2793, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:21<00:00,  7.75it/s]


epoch: 49 loss: tensor(21206.2500, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:21<00:00,  7.73it/s]

epoch: 50 loss: tensor(21260.9121, grad_fn=<DivBackward0>)


In [78]:
distance_matrix = w2v.get_distance_matrix()

In [106]:
similar_words = {search_term: [w2v.data.id2word_dict[idx] for idx in distance_matrix[w2v.data.word2id_dict[search_term]].argsort()[1:10]] 
                   for search_term in ['tablespoon','ballot','mettwurst','pitcher','football','cup']}
similar_words

{'ballot': ['nominating',
  'scatters',
  'larceny',
  'disenfranchised',
  'election',
  'two',
  'troopers',
  'suns',
  'boite'],
 'cup': ['teaspoonful',
  'ketchup',
  'worcestershire',
  'celery',
  'tablespoons',
  'chopped',
  'chives',
  'rub',
  'teaspoon'],
 'football': ['volleyball',
  'letterman',
  'mastodons',
  'university',
  'maladroit',
  'facet',
  'minn',
  'reharmonization',
  'rozelle'],
 'mettwurst': ['bratwurst',
  'bockwurst',
  'knackwurst',
  'cervelat',
  'bologna',
  'lobster',
  'pepperoni',
  'mischief',
  'jerusalem'],
 'pitcher': ['overhand',
  'gentlemanly',
  'ball',
  'haydon',
  'hefty',
  'stop',
  'wryly',
  'ran',
  'away'],
 'tablespoon': ['teaspoons',
  'worcestershire',
  'vinegar',
  'chive',
  'teaspoon',
  'tablespoons',
  'horseradish',
  'chutney',
  'tangy']}